<a href="https://colab.research.google.com/github/avtar123/Face-Recognition/blob/main/Face_Detection_%26_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importing Packages
import os
import cv2
import torch
import pickle
import imutils
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.svm import SVC
from PIL import Image, ImageFilter
from sklearn.pipeline import Pipeline
import torchvision.models as models
from sklearn.pipeline import make_pipeline
import torchvision.transforms as transforms
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import train_test_split
from moviepy.video.io.VideoFileClip import VideoFileClip

In [ ]:
#Creating directory of players
crp_img_dir = "/content/drive/MyDrive/Players_Face_data"
crp_img_dir1 = os.listdir(crp_img_dir)

In [ ]:
#Creating Dictionary for Players
player_names_dict = {}
count = 0
for player in crp_img_dir1:
    player_names_dict.update({"{}".format(count):player})
    count+=1
print(player_names_dict)

{'0': 'virat_kohli', '1': 'Rohit_Sharma', '2': 'Quinton_De_Kock'}


In [ ]:
class_dict = {}
count = 0
for player_id in player_names_dict.keys():
  class_dict[player_id] = count
  count = count + 1
class_dict


{'0': 0, '1': 1, '2': 2}

# Extracting features using HOG

In [ ]:
def hog_feature(img):
  win_size = (64,64)
  block_size = (16,16)
  block_stride = (8,8)
  cell_size = (8,8)
  num_bins = 9
  hog = cv2.HOGDescriptor(win_size,block_size,block_stride,cell_size,num_bins)
  hog_features = hog.compute(img)
  #hog_features = hog_features.flatten()
  hog_features = np.array(hog_features).flatten()
  #hog_img = hog_features.reshape(-1,num_bins)
  scaler = StandardScaler()
  hog_features_normalized = scaler.fit_transform(hog_features.reshape(-1,1))
  return hog_features_normalized

In [ ]:
x_train, y_train, x_test, y_test = [], [], [], []
min_len = 500
for player_id, player_name in player_names_dict.items():
    player_img_dir = os.path.join(crp_img_dir,player_name)
    player_face_img_dir = os.listdir(player_img_dir)
    #print(player_img_dir)
    #print(player_face_img_dir)
    len_dir = len(player_face_img_dir)
    if min_len>len_dir:
      min_len = len_dir
print(min_len)
for player_id, player_name in player_names_dict.items():
  #print(celebrity_name,player_name)
  player_img_dir = os.path.join(crp_img_dir,player_name)
  player_face_img_dir = os.listdir(player_img_dir)
  #print(player_img_dir)
  #print(player_face_img_dir)
  split_counter = 0
  #for player_img in player_face_img_dir[:min_len]:
  for player_img in player_face_img_dir:
    #print(player_img)
    img = cv2.imread(player_img_dir+"/"+player_img)
    #print(img)
    #gray_img = cv2.cvtColor( img,cv2.COLOR_RGB2GRAY )
    #print(gray_img)
    #cv2.imwrite("img.png",gray_img)
    #if gray_img is not None:
    if img is not None:
      #scaled_img = cv2.resize(gray_img, (100,100))
      scaled_img = cv2.resize(img, (100,100))
      #cv2.imwrite("img.png",scaled_img)
      hog_img = hog_feature(scaled_img)
      #cv2.imwrite("img_1.png",hog_img)
      if split_counter<5:
        x_test.append(hog_img)
        y_test.append(class_dict[player_id])
      else:
        x_train.append(hog_img)
        y_train.append(class_dict[player_id])
      #break
    split_counter+=1

110


# Extracting features using VGG

In [ ]:
vgg16 = models.vgg16(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(vgg16.features.children())[:-1])
transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])])

  warnings.warn(

  warnings.warn(msg)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 99.9MB/s]


In [ ]:
x_train, y_train, x_test, y_test = [], [], [], []
min_len = 500
for player_id, player_name in player_names_dict.items():
    player_img_dir = os.path.join(crp_img_dir,player_name)
    player_face_img_dir = os.listdir(player_img_dir)
    #print(player_img_dir)
    #print(player_face_img_dir)
    len_dir = len(player_face_img_dir)
    if min_len>len_dir:
      min_len = len_dir
print(min_len)
for player_id, player_name in player_names_dict.items():
  #print(celebrity_name,player_name)
  player_img_dir = os.path.join(crp_img_dir,player_name)
  player_face_img_dir = os.listdir(player_img_dir)
  #print(player_img_dir)
  #print(player_face_img_dir)
  split_counter = 0
  #for player_img in player_face_img_dir[:min_len]:
  for player_img in player_face_img_dir:
    #print(player_img)
    #img = cv2.imread(player_img_dir+"/"+player_img)
    img = Image.open(player_img_dir+"/"+player_img)
    inp_img = transform(img).unsqueeze(0)
    with torch.no_grad():
      features = feature_extractor(inp_img)
      numpy_features = features.numpy()
    #print(img)


    if features is not None:
      #scaled_img = cv2.resize(gray_img, (64,64))
      #scaled_img = cv2.resize(img, (64,64))
      #cv2.imwrite("img.png",scaled_img)
      #cv2.imwrite("img_1.png",hog_img)
      if split_counter<5:
        x_test.append(numpy_features)
        y_test.append(class_dict[player_id])
      else:
        x_train.append(numpy_features)
        y_train.append(class_dict[player_id])
      #break
    split_counter+=1

130


# Checking shapes of features and converting them accordingly SVM criteria

In [ ]:
print(x_train[0].shape)

(1, 512, 14, 14)


In [ ]:
print(type(x_train[0]))

<class 'numpy.ndarray'>


In [ ]:
x_train = np.array(x_train).reshape(len(x_train),-1)
x_test = np.array(x_test).reshape(len(x_test),-1)

In [ ]:
print(x_train[0].shape)

(100352,)


In [ ]:
print(x_train)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.31122318 0.         0.         ... 3.3831255  0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [ ]:
print(y_train)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [ ]:
print(len(x_train),len(y_train),len(x_test),len(y_test))

315 315 15 15


# **Training**

In [ ]:
pipe = Pipeline([('svc', SVC(kernel = 'poly', C = 10,gamma = "auto"))])
#model = svm.SVC(kernel = 'rbf', C = 10)
pipe.fit(x_train, y_train)
pipe.score(x_test, y_test)

0.6666666666666666

In [ ]:
#Saving the hog feature model
pickle.dump(pipe, open("/content/drive/MyDrive/Face_detect_recog_models/player_recognition__hog_model_73.sav", 'wb'))

In [ ]:
#Saving the VGG feature model
pickle.dump(pipe, open("/content/drive/MyDrive/Face_detect_recog_models/player_recognition__vgg_model_72.sav", 'wb'))

# **Prediction** ***on*** **image**

In [ ]:
#Prediction using HOG features
import hubconf
model = hubconf._create("/content/drive/MyDrive/Face_detect_recog_models/face_detection_model.pt")
pre_img = cv2.imread("/content/rs_54.jfif")
results = np.array(model(pre_img).pred[0].cpu())
results1= np.array(results[np.isin(results[:,5],np.array([0]))])
if len(results)>0:
  for i in results:
    crop_face = pre_img[int(i[1]):int(i[3]),int(i[0]):int(i[2])]
    img_rgb = cv2.cvtColor(crop_face,cv2.COLOR_BGR2RGB)
    pre_gray_img = cv2.cvtColor( img_rgb,cv2.COLOR_RGB2GRAY )
    scaled_pre_gray_img = cv2.resize(pre_gray_img, (100,100))
    pre_hog_img = hog_feature(scaled_pre_gray_img)
    pre_reshape_img = np.array(pre_hog_img).reshape(1,-1)

YOLOv5 🚀 v7.0-210-gdd10481 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [ ]:
#Prediction using HOG features
pre_img = cv2.imread("/content/rohit_sharma_1498.png")
scaled_pre_gray_img = cv2.resize(pre_gray_img, (100,100))
pre_hog_img = hog_feature(scaled_pre_gray_img)
pre_reshape_img = np.array(pre_hog_img).reshape(1,-1)

In [ ]:
#Prediction using VGG
img = Image.open("/content/virat_kohli_1803.png")
inp_img = transform(img).unsqueeze(0)
with torch.no_grad():
  features = feature_extractor(inp_img)
  numpy_features = features.numpy()
pre_reshape_img = np.array(numpy_features).reshape(1,-1)

In [ ]:
#Predicting
load_model = pickle.load(open("/content/drive/MyDrive/Face_detect_recog_models/player_recognition__VGG_model_66.sav", 'rb'))
result = load_model.predict(pre_reshape_img)

In [ ]:
#Name of Player Recognised
player_names_dict["{}".format(result[0])]

'virat_kohli'

# **Prediction** **on** **a** **video**

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-210-gdd10481 Python-3.10.12 torch-2.0.1+cu118 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.0/107.7 GB disk)


In [ ]:
vid = VideoFileClip("/content/drive/MyDrive/Video_data/PUNJAB-KINGS_ROYAL-CHALLENGERS-BANGALORE_20-04-2023_T20_1st_INNING_2_3_FOUR_Virat-Kohli.mp4")
os.chdir("/content/drive/MyDrive/Yolov5")

In [ ]:
def player_recog(img,face_recogniton_model,player_names_dict):

    load_model = pickle.load(open(face_recogniton_model, 'rb'))
    result = load_model.predict(img)
    #result = pipe.predict(img)
    return player_names_dict["{}".format(result[0])]

In [ ]:
#Predicting by HOG feature
import hubconf
model = hubconf._create("/content/drive/MyDrive/Face_detect_recog_models/face_detection_model.pt")
#player_name = os.listdir("/content/drive/MyDrive/Players_face_dataset")
for duration in range(0,int(vid.duration)):

    frame = vid.get_frame(t = duration)
    results = np.array(model(frame).pred[0].cpu())
    results1= np.array(results[np.isin(results[:,5],np.array([0]))])
    if len(results)>0:
        for i in results:
            crop_face = frame[int(i[1]):int(i[3]),int(i[0]):int(i[2])]
            #print(crop_face)
            img_rgb = cv2.cvtColor(crop_face,cv2.COLOR_BGR2RGB)
            #gray_img = cv2.cvtColor( img_rgb,cv2.COLOR_RGB2GRAY )
            scaled_gray_img = cv2.resize(img_rgb, (100,100))
            hog_img = hog_feature(scaled_gray_img)
            final_img = np.array(hog_img).reshape(1,-1)
            Player_name = player_recog(final_img,"/content/drive/MyDrive/Face_detect_recog_models/player_recognition__hog_model_66.sav",player_names_dict)
            print(duration,Player_name)

In [ ]:
#Predicting by VGG feature
import hubconf
model = hubconf._create("/content/drive/MyDrive/Face_detect_recog_models/face_detection_model.pt")
#player_name = os.listdir("/content/drive/MyDrive/Players_face_dataset")
for duration in range(0,int(vid.duration)):

    frame = vid.get_frame(t = duration)
    results = np.array(model(frame).pred[0].cpu())
    results1= np.array(results[np.isin(results[:,5],np.array([0]))])
    if len(results)>0:
        for i in results:
            crop_face = frame[int(i[1]):int(i[3]),int(i[0]):int(i[2])]
            img_rgb = cv2.cvtColor(crop_face,cv2.COLOR_BGR2RGB)
            cv2.imwrite("/content/img.png",img_rgb)
            crop_img = Image.open("/content/img.png")
            inp_img = transform(crop_img).unsqueeze(0)
            with torch.no_grad():
              features = feature_extractor(inp_img)
              numpy_features = features.numpy()
            final_img = np.array(numpy_features).reshape(1,-1)
            Player_name = player_recog(final_img,"/content/drive/MyDrive/Face_detect_recog_models/player_recognition__VGG_model_66.sav",player_names_dict)
            print(duration,Player_name)
            os.remove("/content/img.png")

YOLOv5 🚀 v7.0-210-gdd10481 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


0 Rohit_Sharma
1 Rohit_Sharma
2 Rohit_Sharma
3 Rohit_Sharma
6 Rohit_Sharma
9 Rohit_Sharma
10 Rohit_Sharma
11 Rohit_Sharma
11 Rohit_Sharma
12 Rohit_Sharma
13 Rohit_Sharma
14 Rohit_Sharma
15 Rohit_Sharma
15 Rohit_Sharma
15 Rohit_Sharma
15 Rohit_Sharma
15 Rohit_Sharma
15 Rohit_Sharma
15 Rohit_Sharma


# **Augmenting** **Images**

In [ ]:
count = 0
for player_id, player_name in player_names_dict.items():
  counter = 60
  stopper = 1
  total_img = 130
  print(player_id,player_name)
  player_img_dir = os.path.join(crp_img_dir,player_name)
  player_face_img_dir = os.listdir(player_img_dir)
  #print(player_img_dir)
  #print(player_face_img_dir)
  len_of_player_face_img_dir = len(player_face_img_dir)
  print("No. of images in player's directory: ",len_of_player_face_img_dir)
  req_img = total_img - len_of_player_face_img_dir
  print("Required Images: ",req_img)
  count+=1
  if req_img==0:
    print("No need to increase images!")
  else:
    for player_img in player_face_img_dir:
        #if stopper<10:
        if stopper<=7 and stopper!=(req_img+1):
          print("Stopper: ",stopper)
          print("Enter in IF: ")
          img = cv2.imread(player_img_dir+"/"+player_img)
          #gray_img = cv2.cvtColor( img,cv2.COLOR_RGB2GRAY )
          image = imutils.rotate(img, angle=165)
          #image = cv2.rotate(img, cv2.ROTATE_180) # Try gray scale image with this property
          #image = cv2.rotate(gray_img, cv2.ROTATE_90_CLOCKWISE)
          cv2.imwrite(crp_img_dir+"/"+player_name+"/Image_{}.png".format(counter),image)
          print(crp_img_dir+"/"+player_name+"/img_{}.png".format(counter))
          counter+=1
          stopper+=1
        elif 5<stopper<=13 and stopper!=(req_img+1):
          print("Stopper: ",stopper)
          print("Enter in Elif: ")
          img = cv2.imread(player_img_dir+"/"+player_img)
          #gray_img = cv2.cvtColor( img,cv2.COLOR_RGB2GRAY )
          image = imutils.rotate(img, angle=80)
          #image = cv2.rotate(img, cv2.ROTATE_180) # Try gray scale image with this property
          #image = cv2.rotate(gray_img, cv2.ROTATE_90_CLOCKWISE)
          cv2.imwrite(crp_img_dir+"/"+player_name+"/Image_{}.png".format(counter),image)
          print(crp_img_dir+"/"+player_name+"/img_{}.png".format(counter))
          counter+=1
          stopper+=1
        elif stopper<=req_img:
          print("Stopper: ",stopper)
          print("Enter in Else: ")
          img = cv2.imread(player_img_dir+"/"+player_img)
          #gray_img = cv2.cvtColor( img,cv2.COLOR_RGB2GRAY )
          image = imutils.rotate(img, angle=245)
          #image = cv2.rotate(img, cv2.ROTATE_180) # Try gray scale image with this property
          #image = cv2.rotate(gray_img, cv2.ROTATE_90_CLOCKWISE)
          cv2.imwrite(crp_img_dir+"/"+player_name+"/Image_{}.png".format(counter),image)
          print(crp_img_dir+"/"+player_name+"/img_{}.png".format(counter))
          counter+=1
          stopper+=1
  # if count==2:
  #   break


0 virat_kohli
No. of images in player's directory:  110
Required Images:  20
Stopper:  1
Enter in IF: 
/content/drive/MyDrive/Players_Face_data/virat_kohli/img_60.png
Stopper:  2
Enter in IF: 
/content/drive/MyDrive/Players_Face_data/virat_kohli/img_61.png
Stopper:  3
Enter in IF: 
/content/drive/MyDrive/Players_Face_data/virat_kohli/img_62.png
Stopper:  4
Enter in IF: 
/content/drive/MyDrive/Players_Face_data/virat_kohli/img_63.png
Stopper:  5
Enter in IF: 
/content/drive/MyDrive/Players_Face_data/virat_kohli/img_64.png
Stopper:  6
Enter in IF: 
/content/drive/MyDrive/Players_Face_data/virat_kohli/img_65.png
Stopper:  7
Enter in IF: 
/content/drive/MyDrive/Players_Face_data/virat_kohli/img_66.png
Stopper:  8
Enter in Elif: 
/content/drive/MyDrive/Players_Face_data/virat_kohli/img_67.png
Stopper:  9
Enter in Elif: 
/content/drive/MyDrive/Players_Face_data/virat_kohli/img_68.png
Stopper:  10
Enter in Elif: 
/content/drive/MyDrive/Players_Face_data/virat_kohli/img_69.png
Stopper:  11
Ent

In [ ]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto',probability=True),
        'params' : {
            'svc__C': [1,10,100,1000],
            'svc__kernel': ['rbf','linear','poly','sigmoid']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'randomforestclassifier__n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='lbfgs',multi_class='auto'),
        'params': {
            'logisticregression__C': [1,5,10]
        }
    }
}


# **Testing**

In [ ]:
img1 = cv2.imread("/content/virat_kohli_1865.png")
print(img1.shape)
image = cv2.resize(img1, (200,200))
print(image.shape)
cv2.imwrite("/content/img1.png",image)

In [ ]:
player_id = 0
player_img_dir = []
no_of_images_player = {}
list_dir_path = os.listdir(dir_path)
for player_name in list_dir_path:
  player_img_dir.append(os.path.join(crp_img_dir,player_name))
for player_image_dir in player_img_dir:
  #print(player_image_dir)
  player_images = os.listdir(player_image_dir)
  #print(player_images)
  no_of_images_of_player = len(player_images)
  #print(no_of_images_of_player)
  no_of_images_player.update({"{}".format(player_id):(no_of_images_of_player,player_image_dir.split("/")[-1])})
  #print(no_of_images_player)
  player_id+=1
  #break

In [ ]:
no_of_images_player

In [ ]:
scores = []
best_estimators = {}
for algo, mp in model_params.items():
    pipe = make_pipeline(mp['model'])
    clf =  GridSearchCV(pipe, mp['params'], cv=5, return_train_score=False)
    clf.fit(x_train, y_train)
    scores.append({
        'model': algo,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    best_estimators[algo] = clf.best_estimator_

df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df